In [ ]:
from fastai.vision.all import *
import pandas as pd
import cam
import util

In [ ]:
dls, labels = util.chexpert_data_loader()

In [ ]:
dls.show_batch(max_n=9, figsize=(20,9))

In [ ]:
# First train on conditional probabilities
chexpert_learner_conditional = util.ChexpertLearner(dls, densenet121, n_out=len(labels), y_range=(0, 1),
                                        loss_func=util.BCEFlatHLCP(hierarchy_map=util.hierarchy_map),
                                        metrics=[partial(accuracy_multi, sigmoid=False),
                                                 RocAucMulti(sigmoid=False, average='weighted')])

In [ ]:
chexpert_learner_conditional.find_lr()

In [ ]:
chexpert_learner_conditional.learn_model(use_saved=False , epochs=30, freeze_epochs=2)

In [ ]:
# Next train unconditionally for only transfer learning
chexpert_learner_unconditional = util.ChexpertLearner(dls, densenet121, n_out=len(labels), y_range=(0, 1),
                                        loss_func=BCELossFlat(),
                                        metrics=[partial(accuracy_multi, sigmoid=False),
                                                 RocAucMulti(sigmoid=False, average='weighted')])

In [ ]:
chexpert_learner_conditional.find_lr()

In [ ]:
chexpert_learner_unconditional.learn_model(use_saved=True, train_saved=True, epochs=30, freeze_epochs=30)

In [ ]:
chexpert_learner = chexpert_learner_unconditional
chexpert_learner.learn.show_results()

In [ ]:
interp = Interpretation.from_learner(chexpert_learner.learn)
interp.plot_top_losses(9)

In [ ]:
cam.plot_cam(chexpert_learner.learn)